In [ ]:
'''只用PM2.5作为特征预测'''

In [1]:
# @File    : main.py
# @Date    : 2021-8-6
# @Author  : Langqing Zou
# @Software: VS Code
# @Python Version: python 3.6

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
'''--------------------Read data and Preprocessing--------------------'''  
train_data = pd.read_csv("./train.csv")
temp = pd.read_csv("./train.csv")
del train_data["Date"]
del train_data["stations"]
column = train_data["observation"].unique()

# Try to have more train data:
# Train data: 0-9,1-10,2-11.....; check it with 10,11,12 respectively
features = 18
dataTrain = []
dataLabel = []
temp.drop(['Date', 'stations', 'observation'], axis=1, inplace=True)
for i in range(240): # 240 days
    day = temp[i*features:(i+1)*features]
    # 0-9,1-10,2-11....., 15 sets totally
    for j in range(15):
        everyNineHour = day.iloc[:,j:j + 9] # iloc[row,column]
        label = day.iloc[9,j+9]
        dataTrain.append(everyNineHour)
        dataLabel.append(label)

In [4]:
'''ONLY PM2.5'''
'''--------------------Create model--------------------'''
# Y = b + W1X1 + W2X2 + ...... + W9X9
def linearModel():
    learning_rate = 0.002
    b = 0.0001
    parameters=[0.001]*9 # all the Ws
    loss_history=[]
    dict={0:9,1:9,2:9,3:9,4:9,5:9,6:9,7:9,8:9,9:9}
    iteration_count = 100  #迭代次数
    for i in range(iteration_count):
        # initial bias gradient and weight gradient
        b_grad=0
        w_grad=[0]*9
        loss = 0
        for k in range(3600):
            oneDay = dataTrain[k]
            partsum = b+parameters[0]*float(oneDay.iloc[9,0])+ parameters[1]*float(oneDay.iloc[9,1])+\
                      parameters[2]*float(oneDay.iloc[9,2])+ parameters[3]*float(oneDay.iloc[9,3])+ \
                      parameters[4]*float(oneDay.iloc[9,4])+ parameters[5]*float(oneDay.iloc[9,5])+ \
                      parameters[6]*float(oneDay.iloc[9,6])+ parameters[7]*float(oneDay.iloc[9,7])+ \
                      parameters[8]*float(oneDay.iloc[9,8])- float(dataLabel[k])
            loss = loss + partsum * partsum
            b_grad = b_grad + partsum

            for j in range(9):
                w_grad[j]=w_grad[j]+ partsum * float(day.iloc[dict[j],j % 9])

        loss_history.append(loss/2)
        # update b and w
        b = b - learning_rate * (b_grad/3600)
        for t in range(9):
            parameters[t] = parameters[t] - learning_rate * (w_grad[t]/3600)

    print("finish training")
    return b, parameters

In [5]:
test_data = pd.read_csv('./test.csv')
answer_data = pd.read_csv('./answer.csv')
# print(result_data)
del test_data["a"]
del test_data["b"]
day_data = []
check_data = []
items=18

for i in range(int(len(test_data)/items)):
    day = test_data[i*items:(i+1)*items] # data of a day
    day_data.append(day)

# print(result_data.iloc[239,1])
for j in range(len(answer_data)):
    check_data.append(answer_data.iloc[j,1])

# from model:
b, parameters = linearModel()

predict=[]

for i in range(len(day_data)):
    day=day_data[i]
    p=b+parameters[0]*float(day.iloc[8,0])+parameters[1]*float(day.iloc[8,1])+\
    parameters[2]*float(day.iloc[8,2])+parameters[3]*float(day.iloc[8,3])+\
    parameters[4]*float(day.iloc[8,4])+parameters[5]*float(day.iloc[8,5])+\
    parameters[6]*float(day.iloc[8,6])+parameters[7]*float(day.iloc[8,7])+\
    parameters[8]*float(day.iloc[8,8])
    predict.append(p)

def evaluate(test,predict):
    sum=0
    for i in range(len(predict)):
        sum=sum+(float(test[i])-float(predict[i]))*(float(test[i])-float(predict[i]))
    return sum/len(predict)

print(evaluate(check_data,predict))
print(loss_history[len(loss_history)-1])

NameError: name 'samples' is not defined

In [ ]:
# Finally
# iteration_count = 1000  #迭代次数
# samples = 100
# learning_rate = 0.002
# evaluate = 4.810371555547992e+126
# finally loss: 2.7661168056311173e+127